# Classification task

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import collections
import pydotplus 
import statistics 
import pandas as pd
import os
from datetime import date
from tqdm.notebook import tqdm
from IPython.display import Image  
import scikitplot as skplt
import wittgenstein as lw

from scipy.stats.stats import pearsonr
from scipy.spatial.distance import pdist,  squareform
import scipy.stats as stats
from scipy.stats import randint as sp_randint

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, pairwise_distances, classification_report, confusion_matrix, plot_confusion_matrix # For Model evaluation
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn import tree, metrics
from sklearn.datasets import make_blobs

import seaborn as sns
import re

In [ ]:
# load of the data
DATASET_DIR = "dataset" + os.path.sep
#index_col=False say to not use the first column as ID
df_players = pd.read_csv('players.csv', sep=',', index_col=0) 

In [ ]:
df_players.info()

## Functions

In [ ]:
def discretize_data(dataset, variables): #mapping categorical into numerical
    for variable in variables:
        #get the unique variable's values
        var = sorted(dataset[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        dataset[variable+'_num'] = dataset[variable].map(mapping).astype(int)
    return dataset

In [ ]:
#metrics computed on the test set
def report_scores(test_label, test_pred):
    print(classification_report(test_label, 
                            test_pred, 
                            target_names=['low', 'high']))

## Data for classification

In [ ]:
df_filtered = df_players[['sex', 'hand', 'best_rank','best_rank_points', 'best_of_3_match', 'best_of_5_match',
           'best_of_3_wins', 'best_of_5_wins', 'w_finals', 'tot_minutes', 'sv1st', 'sv1st_win', 'sv2nd_win', 
           'df', 'ace_perc', 'bpS_perc', 'wmatch', 'lmatch', 'nmatch', 'n_tourney']]

In [ ]:
variables = ['sex', 'hand']
df_filtered = discretize_data(df_filtered, variables)

In [ ]:
df_filtered = df_filtered.drop(columns=['sex', 'hand'])

In [ ]:
df_filtered

In [ ]:
threshold = 50
df_filtered.loc[((df_filtered['best_rank']>0) & (df_filtered['best_rank']<=threshold)), 'ranked'] = 1 #high
df_filtered.loc[((df_filtered['best_rank']>0) & (df_filtered['best_rank']>threshold)), 'ranked'] = 0 #low

In [ ]:
df_filtered.loc[((df_filtered['best_rank']>0) & (df_filtered['best_rank']<=threshold))].shape[0] #number of high

In [ ]:
df_classification = df_filtered[df_filtered['best_rank']>0]

In [ ]:
df_classification = df_classification.drop(columns=['best_rank'])

In [ ]:
label = df_classification.pop('ranked')
train_set, test_set, train_label, test_label = train_test_split(df_classification, label, stratify =label, test_size=0.30)

# Classification

### Decision tree

In [ ]:
train_set

In [ ]:
train_label

In [ ]:
dt = tree.DecisionTreeClassifier(criterion='gini', splitter='best', 
                                  max_depth=5, 
                                  min_samples_split=3, min_samples_leaf=4)
dt = dt.fit(train_set, train_label)

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None, 
                         feature_names=list(train_set.columns),  
                         class_names=['low', 'high'],  #[0, 1]
                         filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())

In [ ]:
train_pred_dt = dt.predict(train_set)
test_pred_dt = dt.predict(test_set)

#### evaluation

In [ ]:
dt.predict_proba(train_set)

In [ ]:
#evaulate the accuracy on the train set and the test set
#metrics also contains precision, recall, f1 and the support
print('Accuracy train set ', metrics.accuracy_score(train_label, train_pred_dt))
print('Accuracy test set ', metrics.accuracy_score(test_label, test_pred_dt))
print('Precision train set ', metrics.precision_score(train_label, train_pred_dt, average='weighted'))
print('Recall train set ', metrics.recall_score(train_label, train_pred_dt, average='weighted'))
print('F1 score train set ', metrics.f1_score(train_label, train_pred_dt, average='weighted'))
print('Support train set ', metrics.precision_recall_fscore_support(train_label, train_pred_dt))

In [ ]:
#per il training set
report_scores(train_label, train_pred_dt)

In [ ]:
#per il test set
report_scores(test_label, test_pred_dt)
#l'accuracy è un buon indicatore, è significativa se è maggiore dell'accuracy della majority class. in caso di 
#situazione unbalance anche la precision e la recall aiutano a capire quanti errori abbiamo

In [ ]:
### cross validation

In [ ]:
scores = cross_validate(dt, train_set, train_label, cv=3, return_train_score= True)
print('Fit time ', statistics.mean(scores['fit_time']))
print('Score time ', statistics.mean(scores['score_time']))
print('Test score ', statistics.mean(scores['test_score']))
print('Train score ', statistics.mean(scores['train_score']))

In [ ]:
#compute confusion matrix
cm = confusion_matrix(test_label, test_pred_dt)
cm

In [ ]:
#it is possible to plot the confusion matrix 
plot_confusion_matrix(dt, test_set, test_label)
plt.show() 

In [ ]:
#test_result = test_set
#test_result['ranked'] = test_label
#print classification for pairs of attributes/columns
#sns.pairplot(data = test_result, hue = 'ranked', palette = "Accent")

In [ ]:
#true labels - different colors for different class
plt.scatter(test_set['best_rank_points'].values, test_set['sv1st'].values , c=test_label, s=20);
plt.show()
plt.scatter(test_set['best_of_3_match'].values, test_set['sv1st'].values , c=test_label, s=20);

#### excluded raws

In [ ]:
df_to_rank = df_filtered[df_filtered['best_rank']==0]
df_to_rank = df_to_rank.drop(columns=['best_rank', 'ranked'])

In [ ]:
unknown_pred_dt = dt.predict(df_to_rank)

In [ ]:
df_to_rank['ranked'] = unknown_pred_dt

In [ ]:
df_to_rank[df_to_rank['ranked']==1]

### SVM

In [ ]:
svm = SVC(kernel='sigmoid', C=0.5, gamma='scale', probability=True)
svm.fit(train_set, train_label)

In [ ]:
#prediction on the test test
test_pred_proba_svm = svm.predict_proba(test_set)
test_pred_proba_svm

In [ ]:
test_pred_svm = svm.predict(test_set)

In [ ]:
#compute the performance of the model
report_scores(test_label, test_pred_svm)

In [ ]:
plot_confusion_matrix(svm, test_set, test_label)
plt.show() 

### Rule based

In [ ]:
#we run a grid search to find the best configuration of parameters' values
ripper = lw.RIPPER()
param_grid = {"prune_size": [0.5, 0.6], "k": [1, 3, 5]}
grid_search = GridSearchCV(estimator=ripper, param_grid=param_grid)
grid_search.fit(train_set, train_label, pos_class=1)

In [ ]:
print('Best parameters setting ', grid_search.cv_results_['params'][0])

In [ ]:
#define and fit the rule-based model
#this function requires only one dataset with the labels. 
#To do so, we concatenate the train_set and the train_label
ripper = lw.RIPPER(k=1, prune_size=0.50)
datas = pd.concat([train_set, train_label], axis=1)
ripper.fit(datas, class_feat='ranked', pos_class=1)

In [ ]:
#in this case the model is a set of rules
ripper.out_model()

In [ ]:
ripper_pred = ripper.predict(test_set)
report_scores(test_label, ripper_pred)

In [ ]:
#evaluation of the performance of the classifier
print('Accuracy ', ripper.score(test_set, test_label))
print('Precision ', ripper.score(test_set, test_label, precision_score))
print('Recall ', ripper.score(test_set, test_label, recall_score))

In [ ]:
plot_confusion_matrix(ripper, test_set, test_label)
plt.show() 

In [ ]:
ripper_pred_reasons = ripper.predict(test_set, give_reasons=True)

In [ ]:
indexes = [i for i,elem in enumerate(ripper_pred_reasons[0]) if elem == True]
rules_used = [ripper_pred_reasons[1][elem] for i,elem in enumerate(indexes)]

In [ ]:
rules_used

### Naive Bayes

### Bagging

### Boosting

### AdaBoost

### Random Forest

### Neural Networks

### KNN

# Classification avoiding overfitting